# Proyecto Final Data Science
## Competencia Telstra en Kaggle

In [2]:
# Importar Librerias
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from time import time

In [3]:
# Cargar todas los Data Sets
train = pd.read_csv('datasets/train.csv',index_col=False) #OK
test = pd.read_csv('datasets/test.csv',index_col=False) #OK
event_type = pd.read_csv('datasets/event_type.csv',index_col=False) #OK
log_feature = pd.read_csv('datasets/log_feature.csv',index_col=False) #OK
resource_type = pd.read_csv('datasets/resource_type.csv',index_col=False) #OK
severity_type = pd.read_csv('datasets/severity_type.csv',index_col=False) 


In [4]:
# Pivotear los Logs
## Output: x_log, x_max_vol, x_log_count, x_log_order
x_log = log_feature
x_log.log_feature.replace(' ','_',regex=True,inplace=True)
x_log = x_log.pivot_table(values='volume', index='id', 
                          columns='log_feature', aggfunc=[np.mean]).fillna(0).reset_index()
x_log['vol_sum'] = np.sum(x_log['mean'], axis=1)
x_log['vol_mean'] = np.mean(x_log['mean'], axis=1)

# Encontar el mayor Volume por Id
x_max_vol = log_feature.groupby(['id']).agg({'volume':'max'}).reset_index().rename(columns={'volume':'max_vol'})

#Contar los Logs por Id
x_log_count = log_feature.groupby(['id']).agg({'log_feature':'count'}).reset_index().rename(columns={'log_feature':
                                                                                                     'log_count'})
# Mantener el Orden de los Id's
x_log_order = pd.DataFrame(log_feature['id'].unique()).rename(columns={0:'id'})
x_log_order['log_order'] = np.arange(1,len(x_log_order)+1)


In [5]:
# Ordenar las localizaciones y contarlas
## Output: x_panel, x_location

# Concatenar Train y Test
x_panel = pd.concat([train, test])

# Contar localizaciones
x_location = x_panel.groupby(['location']).agg({'id':'count'}).reset_index().rename(columns={'id':'location_count'})

# Ordenar todo el Dataset (Train+Test) por el Nº de Log
x_panel = pd.merge(x_panel, x_log_order, on='id', how='left', sort=False)
x_panel.sort_values(by=['log_order'], inplace=True)
x_panel.reset_index(drop=True, inplace=True)

# Ordenar cada grupo de Localidades por Id
x_panel['location_order'] = 1
for i in list(x_panel['location'].unique()):
    x = np.arange(1,len(x_panel['location_order'].loc[x_panel['location']==i])+1)
    x_panel['location_order'].loc[x_panel['location']==i] = x
del x


In [6]:
# Pivotear los Eventos
## Output: x_event
x_event = event_type
x_event.event_type.replace(' ','_',regex=True,inplace=True)
x_event = x_event.pivot_table(index='id', columns='event_type', aggfunc=len).fillna(0).reset_index()
x_event['event_count'] = np.sum(x_event[x_event.columns[1:len(x_event.columns)]], axis=1)
x_event.head(5)


event_type,id,event_type_1,event_type_10,event_type_11,event_type_12,event_type_13,event_type_14,event_type_15,event_type_17,event_type_18,...,event_type_50,event_type_51,event_type_52,event_type_53,event_type_54,event_type_6,event_type_7,event_type_8,event_type_9,event_count
0,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [8]:
# Pivotear Recursos
## Output: x_resource
x_resource = resource_type
x_resource.resource_type.replace(' ','_',regex=True,inplace=True)
x_resource = x_resource.pivot_table(index='id', columns='resource_type', aggfunc=len).fillna(0).reset_index()
x_resource['resource_count'] = np.sum(x_resource[x_resource.columns[1:len(x_resource.columns)]], axis=1)
x_resource.head(5)

resource_type,id,resource_type_1,resource_type_10,resource_type_2,resource_type_3,resource_type_4,resource_type_5,resource_type_6,resource_type_7,resource_type_8,resource_type_9,resource_count
0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0
1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
# Pivotear Severidad
## Output: x_severity
x_severity = severity_type
x_severity.severity_type.replace(' ','_',regex=True,inplace=True)
x_severity = x_severity.pivot_table(index='id', columns='severity_type', aggfunc=len).fillna(0).reset_index()
x_severity.head(5)


severity_type,id,severity_type_1,severity_type_2,severity_type_3,severity_type_4,severity_type_5
0,1,1.0,0.0,0.0,0.0,0.0
1,2,0.0,1.0,0.0,0.0,0.0
2,3,1.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,1.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0


In [10]:
# Realizar todos los Joins con el DataSet completo

# x_log
x_panel = pd.merge(x_panel, x_log, on='id', how='left', sort=False)

# x_max_vol
x_panel = pd.merge(x_panel, x_max_vol, on='id', how='left', sort=False)

# x_log_count
x_panel = pd.merge(x_panel, x_log_count, on='id', how='left', sort=False)

# x_location
x_panel = pd.merge(x_panel, x_location, on='location', how='left', sort=False)

# x_event
x_panel = pd.merge(x_panel, x_event, on='id', how='left', sort=False)

# x_resource
x_panel = pd.merge(x_panel, x_resource, on='id', how='left', sort=False)

# x_severity
x_panel = pd.merge(x_panel, x_severity, on='id', how='left', sort=False)


In [11]:
# Realizar Calculos adicionales sobre el DataSet
x_panel_copy = x_panel
x_panel.location.replace('location ','',regex=True,inplace=True)
x_panel['location'] = x_panel['location'].astype(float)
x_panel['location_ratio'] = x_panel['location_order'] / x_panel['location_count']
x_panel['max_proportion_1'] = x_panel['max_vol'] / x_panel['log_count']
x_panel['max_proportion'] = x_panel['max_vol'] / x_panel[('vol_sum', '')]
x_panel['vol_std'] = (((np.log2(((x_panel[('vol_sum', '')] - np.mean(x_panel[('vol_sum', '')]))/np.std(x_panel[('vol_sum', '')]))**2))))
x_panel['f203_d_f82'] = x_panel[('mean','feature_203')] / x_panel[('mean','feature_82')]




In [12]:
# Separar el DataSet nuevamente en Train y Test
x_train = x_panel.loc[~x_panel['fault_severity'].isnull()].reset_index()
x_train.fillna(value = -1, inplace = True)
x_test = x_panel.loc[x_panel['fault_severity'].isnull()].reset_index()
x_test.fillna(value = -1, inplace = True)


In [13]:
# Guardar Data Sets en archivos CSV
x_train.to_csv('py_train_1.csv', encoding='utf-8', index=False)
x_test.to_csv('py_test_1.csv', encoding='utf-8', index=False)

# Paso Final
Se cargan los archivos generados "py_train.csv" y "py_test.csv" a Dataiku. Desde allí se entrenan los modelos para la predicción.